<a href="https://colab.research.google.com/github/seandavi/jupyter_notebooks/blob/main/intro_to_unstructured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install 'unstructured[pdf]'

In [2]:
from unstructured.partition.auto import partition

In [3]:
from urllib.request import urlretrieve

In [4]:
path, headers = urlretrieve('https://www.nature.com/articles/s41598-022-07995-7.pdf','microbiome.pdf')

In [5]:
parts = partition('microbiome.pdf')

[nltk_data] Downloading package punkt to /Users/seandavis/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/seandavis/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [6]:
from collections import defaultdict
categories = defaultdict(int)
for element in parts:
  categories[str(type(element))]+=1
for k, v in categories.items():
  print(k,v)

<class 'unstructured.documents.elements.Title'> 65
<class 'unstructured.documents.elements.NarrativeText'> 84
<class 'unstructured.documents.elements.Text'> 64
<class 'unstructured.documents.elements.ListItem'> 40


## fine-tuning the pdf extraction

In [7]:
from unstructured.partition.pdf import partition_pdf

In [8]:
# Linux
#!sudo apt-get install poppler-utils tesseract-ocr
# MacOS
#!brew install poppler
#!brew install tesseract-ocr

Running `brew update --auto-update`...
==> Auto-updated Homebrew!
Updated 5 taps (slingdata-io/sling, flyteorg/tap, homebrew/services, homebrew/core and homebrew/cask).
==> New Formulae
opensca-cli
==> New Casks
blockstream-green          imazing-profile-editor     salt
egovframedev               sakura                     xact

You have 93 outdated formulae and 3 outdated casks installed.

==> Downloading https://ghcr.io/v2/homebrew/core/poppler/manifests/23.12.0
######################################################################### 100.0%
==> Fetching dependencies for poppler: gettext, glib, libx11, gmp, libtasn1, nettle, p11-kit, openssl@3, libnghttp2, unbound, gnutls, libgpg-error, libassuan, libgcrypt, libksba, libusb, npth, openldap, pinentry, readline, gnupg, python-setuptools, gpgme, xz, nspr and nss
==> Downloading https://ghcr.io/v2/homebrew/core/gettext/manifests/0.22.4
######################################################################### 100.0%
==> Fetching gettext
=

In [9]:
parts2 = partition_pdf(filename = "microbiome.pdf",
                                 # Unstructured first finds embedded image blocks
                                 extract_images_in_pdf=False,
                                 # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
                                 # Titles are any sub-section of the document
                                 infer_table_structure=True,
                                 # Post processing to aggregate text once we have the title
                                 chunking_strategy="by_title",
                                 # Chunking params to aggregate text blocks
                                 # Attempt to create a new chunk 3800 chars
                                 # Attempt to keep chunks > 2000 chars
                                 max_characters=4000,
                                 new_after_n_chars=3800,
                                 combine_text_under_n_chars=2000,
                                 #image_output_dir_path=path
                                 )

In [10]:
from collections import defaultdict
categories = defaultdict(int)
for element in parts2:
  categories[str(type(element))]+=1
for k, v in categories.items():
  print(k,v)

<class 'unstructured.documents.elements.CompositeElement'> 24
<class 'unstructured.documents.elements.Table'> 6


In [11]:
lens = list([len(x.text) for x in parts2])

## ChromaDB

In [16]:
#%%capture
%pip install --upgrade chromadb


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: /Users/seandavis/.pyenv/versions/3.11.1/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
# prompt: import chromadb and load the documents in the parts2 list

import chromadb
client = chromadb.Client()
client.delete_collection('my_collection')
collection = client.create_collection(name="my_collection")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/seandavis/.pyenv/versions/3.11.1/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/s3/5l6g4_rj4q9923t7hw7k2lpr0000gq/T/ipykernel_86032/387620397.py", line 4, in <module>
    client = chromadb.Client()
             ^^^^^^^^^^^^^^^^^
  File "/Users/seandavis/.pyenv/versions/3.11.1/lib/python3.11/site-packages/chromadb/__init__.py", line 92, in Client
ModuleNotFoundError: No module named 'chromadb.api.local'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/seandavis/.pyenv/versions/3.11.1/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/seandavis/.pyenv/versions/3.11.1/lib/python3.11/site-packages/IPyth

In [ ]:
collection.add(documents=[x.text for x in parts2], metadatas = [{'filename': x.metadata.filename} for x in parts2], ids=list(map(lambda x: str(x), range(len(parts2)))))

In [ ]:
res = collection.query(
    query_texts=["What is the statistical model used for analysis?"],
    n_results=10,
)

In [ ]:
res.keys()